# 2水準系直交表への自動割付け(+実験計画の作成)
---
● 今回のPython実装内容  
- 2水準直交表への因子と交互作用の割付(わりつけ)を行います。  
- 割付後の表を実験計画としてEXCEL出力します。

今回、直交表はL4,L8,L16,L32から自動選択します。  
（手持ちの表がL4～16までだったため、どなたかL32以上で成分記号も書かれている表をお持ちでしたらいただけないでしょうか。）

### import

In [3]:
!pip install --upgrade pip
!pip install openpyxl

ERROR: Could not find a version that satisfies the requirement openpyxl (from versions: none)
ERROR: No matching distribution found for openpyxl


In [5]:
'''
ver.3 交互作用が0個の時の処理を追加
'''

# import
import sys
import pandas as pd
import math
import itertools  # permutation(順列)の作成用
#import openpyxl   # excel出力用
print("Python version : ",sys.version)
print("pandas version : ",pd.__version__)
#print("openpyxl version : ",openpyxl.__version__)

Python version :  3.7.7 (tags/v3.7.7:d7c567b08f, Mar 10 2020, 10:41:24) [MSC v.1900 64 bit (AMD64)]
pandas version :  1.3.5


### 初期設定

In [8]:

# 因子の名前
factor_symbols_all = ["時間", "小麦", "カレー粉"]    # ["A","B","C","D","F","G","H","I","J","K","L","M","N","O","P"]

# 各因子の水準
factors = [
    [20,60],
    [5,10],
    ["海軍","BMT"]
]

# 見たい交互作用を因子の番号(o～)で指定。
interactions = [
     [0, 1],    # AxB
     [1, 2]     # BxC
]

# 最後に実験計画の順序をランダマイズするか設定
use_randomize = True
use_seed = True  # Falseの場合、毎回変化する
seed_no = 0  # SEEDを指定(int?)

### 結果表示用の文字列を作成

In [9]:
## 因子
factor_symbols = [""] * len(factors)
for i,fct in enumerate(factors):
    factor_symbols[i] = factor_symbols_all[i]
print("因子",factor_symbols)

## 交互作用    
interaction_symbols = [""] * len(interactions)
for i,itr in enumerate(interactions):
    text = factor_symbols[itr[0]] + "x" + factor_symbols[itr[1]]
    interaction_symbols[i] = text
print("交互作用",interaction_symbols)

因子 ['時間', '小麦', 'カレー粉']
交互作用 ['時間x小麦', '小麦xカレー粉']


# 割付け
---

### 直交表サイズ推定
今回は2水準の直交表への割付を行います。   
因子と相互作用の数から、使用する直交表をL4,L8,L16から選択します。  
また、後ほど使用する、因子の割付け優先順位のリストを準備します。

※注意：
　この順位にした理由は「成分記号による割付け法」で説明できるのですが、今回説明は割愛しています。  
　このリストの作成理由は、このリストに沿って割り付けることで割付けの試行回数を減らすことと、    
　人間（私個人）が割付けた場合に近い表を作るためです。ですので一部、表の因子並び順には個人差が出る場合もあります。  
　※※あくまで表の見た目の問題なので、結果や解析には影響しません！

In [10]:
# 例：L8で行けそうだぞと判定
# factor(因子)は4つ,interaction(交互作用)は2つ,合計6なので、L8(2^7)で行けそう
l4 = ["*"] * 3
l8 = ["*"] * 7
l16 = ["*"] * 15
l32 = ["*"] * 32

length = len(factors) + len(interactions)
print("length= ",length)

if length <= 3:
    ls = l4
    yusen_list = [1,2,3]  # 因子割付の優先順位(仮), 成分記号1つの列は1,2
elif length <= 7:
    ls = l8
    yusen_list = [1,2,4,7,6,5,3]  # 因子割付の優先順位(仮) 成分記号1つの列は1,2,4
elif length <= 15:
    ls = l16
    yusen_list = [1,2,4,8,15,14,13,11,7,12,10,9,6,5,3]  # 因子割付の優先順位(仮) 成分記号1つの列は1,2,4,8
elif length <= 31:
    ls = l32
    yusen_list = [1,2,4,8,16,31,30,29,28,27,26,25,24,23,22,21,20,19,18,17,15,14,13,11,7,12,10,9,6,5,3]  # 因子割付の優先順位(仮、成分記号不明なのでやや適当。。) 成分記号1つの列は1,2,4,8,16
else :
    print("因子、または交互作用が多すぎます。")
    quit()
    
seibun_1keta_count = int(math.log(len(ls)+1,2))
print("成分記号1桁の数は",seibun_1keta_count)

print("\nL"+str(len(ls)+1)+"直交表に割付け処理を行います")

length=  5
成分記号1桁の数は 3

L8直交表に割付け処理を行います


### 交互作用の表による割付けなど、関数で定義
交互作用の表には、
- 例「1列と2列の交互作用⇒3列に現れる」

の様に、「全ての列同士の組み合わせ」と「交互作用が表れる列」の対応が書かれています。  
この表を利用して、機械的に交互作用の列を特定します。

その他関数として、
- リスト同士に重複が含まれるかの判定
- リスト同士の重複部分を抽出

を定義しています。

In [11]:
def itrs_allocate(fcts,itrs):  # fctsには、相互作用に使われる因子の列番が入っていること
    # 交互作用の表から、交互作用の列番を判定
    filepath = "interaction_table.csv"
    df_itrs_table = pd.read_csv(filepath)
    # df_itrs_table
    # interaction_1について、Aは1列、Bは2列⇒ 3列だ
    # 3列が使われていないか確認⇒設定
    # interaction_2について、Aは1列、Cは4列⇒ 5列だ
    itrs_retsuban = [0]*len(itrs)
    for itr_i in range(len(itrs)):
        row = fcts[itrs[itr_i][0]] - 1
        col = fcts[itrs[itr_i][1]]
#         print("row=",row)
#         print("col=",col)
        itrs_retsuban[itr_i] = df_itrs_table[str(col)].iloc[row]
#     print("交互作用の割付",itrs_retsuban) 
    return itrs_retsuban

# listに重複があればTrueを返す
def has_duplicates(seq):
    return len(seq) != len(set(seq))

# 2つのlistの重複チェック。一つでも重複していればTrueを返す
def has_duplicates_2lists(list_a,list_b):
    duplicate_is_exist = False
    for a in list_a:
        if a in list_b:
            duplicate_is_exist = True
    return duplicate_is_exist

# 2つのlistの重複をリストで返す
def extract_duplicates_2lists(list_a,list_b):
    duplicates = []
    for a in list_a:
        if a in list_b:
            duplicates.append(a)
    return duplicates
    

### 割付け(MAIN)
交互作用を見る/見ないで、割付の手順が変わります。
- 交互作用無しの場合、因子を優先順位リスト順に割付け
- 交互作用有りの場合、交互作用で見たい因子の組み合わせを作成し、優先順位リスト順に割付ける。  
そして交互作用が表れる列を「交互作用の表」から導き出し、  
  - 先程割り付けた列との重複がなければ確定。
  - 重複あれば、因子の割付け組み合わせを変更してやり直し。
    - 因子と交互作用に重複がなくなるまで繰り返し。
  
因子の組み合わせはPermutation（順列）で作成します。


In [12]:
fcts_retsuban = [0]*len(factors)      # 因子の列番を格納する変数
itrs_retsuban = [0]*len(interactions) # 交互作用の列番を格納する変数

# 成功/失敗の最終判定用
solution_is_exist = False

# 見たい交互作用が0個の場合
if len(interactions) == 0:
    print("交互作用なし")
    for fct_i in range(len(factors)):
        fcts_retsuban[fct_i] = yusen_list[fct_i]
    solution_is_exist = True
    print("因子の割付",fcts_retsuban)

# 交互作用がある場合
else:
    # 相互作用で使われている因子を判定
    used_fcts_in_itrs = []
    for i in range(len(interactions)):
        used_fcts_in_itrs.extend(interactions[i])  # 相互作用で使われているリストを合体
    used_fcts_in_itrs = pd.DataFrame(used_fcts_in_itrs)  # DataFrameへ変換
    used_fcts_in_itrs = used_fcts_in_itrs[0].unique()    # 重複を削除
    print("交互作用に使用される因子を抽出", used_fcts_in_itrs)

    # 交互作用で使用されていない因子を判定
    surplus_fcts = []
    for fct_i in range(len(factors)):
        if not fct_i in used_fcts_in_itrs:
            surplus_fcts.append(fct_i)
    print("交互作用に使用されない因子を抽出", surplus_fcts)

    # 相互作用に使われている因子の数が、成分記号が一つの数以下の場合。
    if len(used_fcts_in_itrs) <= seibun_1keta_count:
        yusen_i = 0
        for fct_i in range(len(factors)):
            if fct_i in used_fcts_in_itrs:
                fcts_retsuban[fct_i] = yusen_list[yusen_i]
                yusen_i += 1
        print("交互作用に使用される因子の仮割付",fcts_retsuban)

        # 交互作用の仮割付
        itrs_retsuban = itrs_allocate(fcts_retsuban,interactions)
        print("交互作用の仮割付",itrs_retsuban)

        # 重複確認
        if has_duplicates_2lists(itrs_retsuban, fcts_retsuban):
            dups = extract_duplicates_2lists(itrs_retsuban, fcts_retsuban)
            is_dup_list = [False] * len(itrs_retsuban)
            for i,retsuban in enumerate(itrs_retsuban):
                if retsuban in dups:
                    is_dup_list[i] = True
            print("重複あり",itrs_retsuban,"=",is_dup_list,"\n")
        elif has_duplicates(itrs_retsuban) :
                print("交互作用内に重複あり\n")
        else :
            solution_is_exist = True
            print("                                       => 重複なし\n")

    # 交互作用に使われている因子の数が、成分記号が一つの数より多い場合、複数パターンで確認
    else:
        # 成分記号が一つの数を優先で割付け
        for i in range(seibun_1keta_count):
            fcts_retsuban[used_fcts_in_itrs[i]] = yusen_list[i]
        print("成分記号が一つの数を優先で割付け",fcts_retsuban)

        # 成分記号が2桁以上の因子の順列を作成
        surplus_list = yusen_list[seibun_1keta_count:]  # surplus = 余り
        print("未割付の列番",surplus_list,"\n")
        over_count = len(used_fcts_in_itrs) - seibun_1keta_count  # 未割付の、交互作用で使用される因子の数
        ps = itertools.permutations(surplus_list,over_count)  # 順列を作成

        # 順列の数だけ繰り返し
        for p in ps:
            yusen_i = seibun_1keta_count
            for fct_i in range(len(p)):
    #             print("p[" + str(fct_i) + "] = ",p[fct_i])
    #             print(used_fcts_in_itrs[yusen_i])
                fcts_retsuban[used_fcts_in_itrs[yusen_i]] = p[fct_i]
                yusen_i += 1
            print("交互作用に使用される因子の仮割付",fcts_retsuban)

            # 交互作用の割付
            itrs_retsuban = itrs_allocate(fcts_retsuban,interactions)
            print("交互作用の仮割付",itrs_retsuban) 

            # 重複の確認
            if has_duplicates_2lists(itrs_retsuban, fcts_retsuban):
                dups = extract_duplicates_2lists(itrs_retsuban, fcts_retsuban)
                is_dup_list = [False] * len(itrs_retsuban)
                for i,retsuban in enumerate(itrs_retsuban):
                    if retsuban in dups:
                        is_dup_list[i] = True
                print("重複あり",itrs_retsuban,"=",is_dup_list,"\n")
            elif has_duplicates(itrs_retsuban) :
                print("交互作用内に重複あり\n")
            else :
                print("                                       => 重複なし\n")
                solution_is_exist = True
                # 解が見つかり次第、検索を中断する。 TODO:複数の解検索もしたくなるかも
                break
            
if solution_is_exist:
    print("割付成功")
    # 割付の正誤判定  TODO:成分記号による検算を後ほど実装します
    
else:
    print("割付失敗")


交互作用に使用される因子を抽出 [0 1 2]
交互作用に使用されない因子を抽出 []
交互作用に使用される因子の仮割付 [1, 2, 4]
交互作用の仮割付 [3, 6]
                                       => 重複なし

割付成功


### ここまでの結果表示

In [13]:
if solution_is_exist:

    # 割付の正誤判定（今回はやらない）　TODO:成分記号の計算から正誤を検算するプログラムを作成予定。

    # L*に割り付けました
    print("\nResult：割付成功\n\nL"+str(len(ls)+1)+"直交表に割付け処理を行いました。\n")

    print("因子",factor_symbols)
    print("交互作用",interaction_symbols)
    
    # 最終的な割付けリストを作成
    for fct_i,fct_retsu in enumerate(fcts_retsuban):
        if fct_retsu != 0:
            ls[fct_retsu-1] = factor_symbols[fct_i]
    if len(interactions)>0:
        for itr_i,itr_retsu in enumerate(itrs_retsuban):
            ls[itr_retsu-1] = interaction_symbols[itr_i]
#     print("割付結果")
#     retsubans = [0] * len(ls)
#     for i in range(len(ls)):
#         retsubans[i] = i + 1
#     print("列番：", retsubans)
    print("割付：", ls)
    
    # 余りの因子も文字で表現
    if len(interactions) > 0 and len(surplus_fcts) > 0:
        surplus_symbols = [""] * len(surplus_fcts) 
        for fct_i,fct_retsu in enumerate(surplus_fcts):
            surplus_symbols[fct_i] = factor_symbols[surplus_fcts[fct_i]]
        print("余り因子(任意の列番に割付)",surplus_symbols)


Result：割付成功

L8直交表に割付け処理を行いました。

因子 ['時間', '小麦', 'カレー粉']
交互作用 ['時間x小麦', '小麦xカレー粉']
割付： ['時間', '小麦', '時間x小麦', 'カレー粉', '*', '小麦xカレー粉', '*']


### 余り因子と誤差の割付け
交互作用ありの時、交互作用に使用されなかった因子の割付けを行います。
- ここでは、直交表の右側から割付けとしました。この並び順も、結果には影響しませんが個人差があります。

In [14]:
# 直交表の列に未割付の"*"があれば
if "*" in ls:
    # 余りの因子があれば
    if len(interactions) > 0 and len(surplus_fcts) > 0:
        # 一番末尾の"*"から、余り因子に変更
        surplus_count = len(surplus_fcts)
        ls_count = len(ls)
        for i, l in enumerate(reversed(ls)):
            reversed_i = ls_count - i - 1
            if l == "*":
                if surplus_count>0:
                    ls[reversed_i] = factor_symbols[surplus_fcts[surplus_count - 1]]
                    fcts_retsuban[surplus_fcts[surplus_count - 1]] = reversed_i + 1
                    surplus_count -= 1
                else:
                    ls[reversed_i] = "e"  # 最後に残った"*"にはe（誤差)を割付け
    else:
        for i,l in enumerate(ls):
            if l == "*":
                ls[i] = "e"  # 最後に残った"*"にはe（誤差)を割付け
        
    print("因子の割付", fcts_retsuban)
    if len(interactions) > 0 :
        print("交互作用の割付", itrs_retsuban)
    print("全ての割付", ls)
else:
    print("余りの因子無し")
        

因子の割付 [1, 2, 4]
交互作用の割付 [3, 6]
全ての割付 ['時間', '小麦', '時間x小麦', 'カレー粉', 'e', '小麦xカレー粉', 'e']


# 直行表（実験計画）の作成
---
さて、因子と交互作用の割付けが完了しましたので、残りは直交表(実験計画)の作成です。  
ここでは機械的に、直交表の1,2をそれぞれの因子の水準1と水準2に置換します。  

また、実験の順序による影響を小さくするため、実験順序のランダム化を行います。

最後に、結果書き込み用の列と、備考書き込み用の列も空欄で作成しておきます。  

※備考は後から見返す際に有効なことが多いので、積極的に書くと良いです。  
　たとえば「実験5回目と6回目の間に30分休憩を挟んだ」など、細かいことでも書いておきます。


### 直交表の判定、元の表を外部から読み込み

In [15]:
# 直交表のサイズを判定
L = len(ls) + 1
if L == 4:
    l_filepath = "L4.csv"
elif L == 8:
    l_filepath = "L8.csv"
elif L == 16:
    l_filepath = "L16.csv"
else:
    print("割付が不正です")

# 直交表の読み込み
print("直交表の元データ：",l_filepath)
df = pd.read_csv(l_filepath, index_col =0)
print("直交表：",df)
df = df.dropna()
df = df.astype(int)
df

直交表の元データ： L8.csv
直交表：              1    2    3    4    5    6  7
L8                                        
1            1    1    1    1    1    1  1
2            1    1    1    2    2    2  2
3            1    2    2    1    1    2  2
4            1    2    2    2    2    1  1
5            2    1    2    1    2    1  2
6            2    1    2    2    1    2  1
7            2    2    1    1    2    2  1
8            2    2    1    2    1    1  2
component    a  NaN    a         a  NaN  a
NaN        NaN    b    b  NaN  NaN    b  b
NaN        NaN  NaN  NaN    c    c    c  c


,1,2,3,4,5,6,7
L8,,,,,,,
1,1,1,1,1,1,1,1
2,1,1,1,2,2,2,2
3,1,2,2,1,1,2,2
4,1,2,2,2,2,1,1
5,2,1,2,1,2,1,2
6,2,1,2,2,1,2,1
7,2,2,1,1,2,2,1
8,2,2,1,2,1,1,2


### 直交表への水準の記入

In [16]:
row_count = len(df)

# 一度、直交表の1,2⇒_True_,_False_(文字列)に変換。置換間違いを防ぐため
for fct_i in range(row_count - 1):
    str_i = str(fct_i + 1)
    df[str_i] = df[str_i].replace(1,"_True_")
    df[str_i] = df[str_i].replace(2,"_False_")
    
# 直交表への水準の置換
for fct_i,retsuban in enumerate(fcts_retsuban):
    retsuban_str = str(retsuban)
    df[retsuban_str] = df[retsuban_str].replace("_True_",factors[fct_i][0])
    df[retsuban_str] = df[retsuban_str].replace("_False_",factors[fct_i][1])

# 残りを空欄に置換
for fct_i in range(row_count - 1):
    retsuban_str = str(fct_i + 1)
    if not fct_i + 1 in fcts_retsuban:
        df[retsuban_str] = ""

# 表の項目名を入れる
df.columns = ls

# 表に結果、備考欄を追加
df["結果"] = ""
df["備考"] = ""

df

,時間,小麦,時間x小麦,カレー粉,e,小麦xカレー粉,e,結果,備考
L8,,,,,,,,,
1,20,5,,海軍,,,,,
2,20,5,,BMT,,,,,
3,20,10,,海軍,,,,,
4,20,10,,BMT,,,,,
5,60,5,,海軍,,,,,
6,60,5,,BMT,,,,,
7,60,10,,海軍,,,,,
8,60,10,,BMT,,,,,


In [17]:
# 乱数で並べ替え
if use_randomize:
    l_index = range(1, row_count + 1)
    df_index = pd.Series(l_index)
    df["行No."] = df.index
    if use_seed:
        df = df.sample(frac=1, random_state=seed_no) # SEED固定
    else:
        df = df.sample(frac=1) # SEED固定なし
    df.index = df_index

df

,時間,小麦,時間x小麦,カレー粉,e,小麦xカレー粉,e,結果,備考,行No.
1,60,10,,海軍,,,,,,7
2,20,10,,海軍,,,,,,3
3,20,5,,BMT,,,,,,2
4,60,10,,BMT,,,,,,8
5,20,10,,BMT,,,,,,4
6,20,5,,海軍,,,,,,1
7,60,5,,BMT,,,,,,6
8,60,5,,海軍,,,,,,5


## EXCEL出力
作成した実験計画をEXCELに保存します。  
この時、後ほど解析時に読み込む用の情報を別シートに記入しておきます。

In [18]:
excel_name = "TEST_NAKAZAWA_output_Expt_Plan_L" + str(L) + ".xlsx"

df_fcts = pd.DataFrame(factors)  # 因子
df_itrs = pd.DataFrame(interactions)  # 交互作用
df_fcts_retsuban = pd.Series(fcts_retsuban)
df_itrs_retsuban = pd.DataFrame(itrs_retsuban)
df_fcts = pd.concat([df_fcts,df_fcts_retsuban], axis=1)
df_fcts.columns = [1,2,"Col_Index"]
if len(df_itrs) != 0:
    df_itrs = pd.concat([df_itrs,df_itrs_retsuban], axis=1)
    df_itrs.columns = [1,2,"Col_Index"]

# データの書き込み
with pd.ExcelWriter(excel_name) as writer:
    df.to_excel(writer,sheet_name='Expt_Plan')  # 作成した直交表（実験計画）
    df_fcts.to_excel(writer,sheet_name='Factors')  # 因子
    df_itrs.to_excel(writer,sheet_name='Interactions')  # 交互作用

# excelのシート非表示化
# wb = openpyxl.load_workbook(excel_name)
# ws_fcts = wb['factors']
# ws_fcts.sheet_state = 'hidden'
# ws_itrs = wb['interactions']
# ws_itrs.sheet_state = 'hidden'
wb.save(excel_name)

print("出力完了",excel_name)

ModuleNotFoundError: No module named 'openpyxl'

## まとめ、所感
- 2水準系直交表への因子と交互作用の自動割付けを行いました。  
- 業務の都合でexcelかPython環境のため、今回はPythonで作成しました。
- Rには、すでに割付け可能なライブラリがあるそうです。
  - Rに詳しい人に聞いてみたいです（Rに触ったことがなく、また周りにも経験者がおらず、、今後勉強します。）
  
## 今後の課題
- 割付け方法に「交互作用の表を使った方法」と「成分記号による方法」  
の2通りを行い、検算したいと考えています。  
※現状は「交互作用の表」のみで割付けています。
- 3水準への対応
- 4水準、擬水準への対応